In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.metrics import log_loss

In [2]:
os.chdir('/aichallenge')

In [138]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
train_imputed = pd.read_csv('train_imputed.csv')
test_imputed = pd.read_csv('test_imputed.csv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


178.45411562919617


In [139]:
train.shape

(250000, 237)

In [140]:
train_imputed.isnull().any().any()

False

In [141]:
test_imputed.shape

(1888195, 42)

In [142]:
train_imputed.shape

(226140, 42)

In [144]:
train_imputed = train_imputed.drop('Unnamed: 0',axis=1)
test_imputed = test_imputed.drop('Unnamed: 0',axis=1)

In [145]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0013070106506347656


In [147]:
train['renewed_yorn'].isnull().sum()

23860

In [148]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.4141356945037842


In [149]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

4.526135444641113


In [150]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

48.96603035926819


In [151]:
len(unclean_columns)

36

In [152]:
len(clean_columns)

133

In [153]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [154]:
len(target_columns)

66

In [155]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.817573070526123


In [156]:
# 56 messy columns are removed
len(messy_columns)

25

In [157]:
len(train_missing.columns)

66

In [158]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

0.934232234954834


In [159]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.8801157474517822


In [160]:
len(messy_columns)

61

In [161]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.2246792316436768


In [162]:
train.shape

(226140, 72)

In [167]:
train_missing.shape

(226140, 41)

In [168]:
train_imputed.shape

(226140, 41)

In [169]:
train_imputed.isnull().any().any()

False

In [170]:
train.isnull().any().any()

False

In [171]:
train_copy=train

In [172]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.2853353023529053


In [198]:
train=train_copy

In [199]:
train.isnull().any().any()

True

In [201]:
train=train.reset_index()

In [213]:
train = train.drop(['index','innovation_challenge_key'],axis=1)

In [203]:
for col in train_imputed.columns:
    train[col]=train_imputed[col]
    test[col]=test_imputed[col]

In [204]:
train[col].shape

(226140,)

In [205]:
train.shape

(226140, 114)

In [206]:
test.shape

(1888195, 112)

In [207]:
train.isnull().any().any()

False

In [215]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.02973771095275879


In [216]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.006474018096923828


In [210]:
train.isnull().any().sum()

0

In [211]:
test.isnull().any().any()

False

In [217]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

2.2494313716888428


In [218]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05
mean,1.005455e-17,2.815275e-17,-2.915821e-17,4.775913e-18,3.016366e-18,5.027277e-18,-9.426144e-20
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01,-2.737838e+00,-1.742868e-01,-7.790595e-02,-1.128741e+00,-5.226844e-02
25%,-1.252316e-01,-2.321716e-01,-6.842019e-01,-1.053205e-01,-7.719748e-02,-2.906674e-01,-2.610574e-02
50%,-1.252316e-01,-2.321716e-01,-7.744580e-02,-4.824491e-02,-7.436361e-02,-1.738249e-02,-1.613900e-02
75%,-1.252316e-01,-1.584198e-01,2.959425e-01,6.452532e-03,-6.515353e-02,1.283695e-01,-1.188889e-03
max,7.774224e+01,6.648806e+01,4.118197e+01,9.915151e+01,1.359952e+02,4.662324e+01,1.670925e+02


In [219]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [220]:
for col in train.columns:
    print(col+' '+str(len(train[col].unique())))

minor_line_yorn 2
instance_status 1
product_transaction_type 3
installed_product_type 2
installed_product_component_yorn 2
component_type 4
scdc_product_yorn 1
service_classification 2
contract_line_multi_year_discount_yorn 2
contract_line_route_to_market_code 3
contract_line_sales_motion_code 1
default_multi_year_discount_yorn 2
low_dollar_contract_yorn 2
contract_line_source_business_process_name 2
contract_line_repair_service_order_level 2
contract_line_reaction_time_code 3
product_route_to_market 5
service_route_to_market_code 5
auto_quote_yorn 2
hardware_yorn 2
software_yorn 2
acw_yorn 2
enterprise_license_agreement_yorn 2
software_opportunity_type 3
sales_hierarchy_level 6
united_states_yorn 2
level_1_name 5
service_sales_node_base_sales_hierarchy_level 6
service_sales_node_base_scms_name 6
service_sales_node_base_united_states_yorn 2
service_sales_node_base_country_type 3
service_sales_node_base_level_1_name 6
item_type 5
manufacturing_plant 2
software_stack 4
monetization_type 

In [221]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

20.592148065567017


In [222]:
train.shape

(226140, 282)

In [223]:
test.shape

(1888195, 282)

In [224]:
train.isnull().any().any()

False

In [225]:
test.isnull().any().any()

False

In [226]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.4516303539276123


In [227]:
X_train.shape

(151513, 282)

In [29]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name

In [228]:
import gc
del dataset
gc.collect()

1707

In [229]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier(verbose=True)
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()
t2-t1

83.12609124183655

In [230]:
pred = xgb1.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))

0.5498826741148257


In [33]:
clean_col_name=[]
for col in test.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
test.columns = clean_col_name
clean_col_name=[]

In [231]:
pred=xgb1.predict_proba(test)
submit=pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('xgb_all_col_modelfit.csv',index=False)

In [232]:
submit.shape

(1888195, 2)

In [233]:
submit.iloc[1]

INNOVATION_CHALLENGE_KEY    6.000000
RENEWAL_PROBABLIITY         0.733574
Name: 139748, dtype: float64

In [245]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
        'booster' : 'gbtree',
        'max_depth' : int(max_depth),
        'gamma' : gamma,
        'eta' : 0.01,
        'objective' : 'binary:logistic',
        'nthread' : 4,
        'silent' : True,
        'eval_metric': 'logloss',
        'subsample' : max(min(subsample, 1), 0),
        'colsample_bytree' : max(min(colsample_bytree, 1), 0),
        'min_child_weight' : min_child_weight,
        'max_delta_step' : int(max_delta_step),
        'seed' : 1001
        }

    folds = 5
    cv_score = 0

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 20,
                    metrics = 'logloss',
                    show_stdv = True
               )

    val_score = xgbc['test-logloss-mean'].iloc[-1]
    train_score = xgbc['train-logloss-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)
    return (val_score*2) - 1

In [246]:
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])

In [247]:
from bayes_opt import BayesianOptimization
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 50.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [248]:
t1=time.time()
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.516889 val-auc = 0.520559 ( diff = -0.003669 ) train-gini = 0.033779 val-gini = 0.041117
    1 | 16m51s |    0.04112 |             0.8476 |   35.3278 |           7.5511 |      7.4613 |             3.3223 |      0.8615 | 
 Stopped after 500 iterations with train-auc = 0.484062 val-auc = 0.490837 ( diff = -0.006774 ) train-gini = -0.031875 val-gini = -0.018326
    2 | 14m08s |   -0.01833 |             0.4605 |   28.0920 |           7.4183 |     11.6498 |             5.0370 |      0.8461 | 
Bayesian Optimization
---------------------------

In [249]:
XGB_BO.res['max']['max_val']

0.15257159999999992

In [250]:
XGB_BO.res['max']['max_params']

{'colsample_bytree': 1.0,
 'gamma': 50.0,
 'max_delta_step': 1.6782624098236996e-09,
 'max_depth': 2.0,
 'min_child_weight': 20.0,
 'subsample': 0.40000000057157475}

In [251]:
m_params = XGB_BO.res['max']['max_params']

In [252]:
t1=time.time()
xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
t2-t1

74.71331357955933

In [253]:
print(log_loss(y_valid,pred[:,1]))

0.6931471824645996


In [254]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-BO-model_imputed.csv',index=False)

In [255]:
t1=time.time()
xgb1=xgb.XGBClassifier(max_depth=int(7.4613),gamma=35.3278,colsample_bytree=0.8476,
                       eval_metric='logloss',max_delta_step= 7.5511,min_child_weight=3.3223,
                      subsample= 0.8615)
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.4958066490196161


169.23563814163208